<a href="https://colab.research.google.com/github/ThousandAI/pycs4001/blob/main/class09/kaggle_dc_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Titanic Decision Tree**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **讀取資料**

In [ ]:
train_data = pd.read_csv("https://raw.githubusercontent.com/ThousandAI/pycs4001/main/class09/train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/ThousandAI/pycs4001/main/class09/test.csv")
test_ids = test_data["PassengerId"]

In [ ]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)

    #data.Embarked.fillna("U", inplace=True)

    return data

In [ ]:
train_data = clean(train_data)
test_data = clean(test_data)

In [ ]:
train_data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [ ]:
X = train_data.iloc[:, 1:]
Y = train_data.iloc[:, 0]
print(f"train data shape: {X.shape}")
print(f"test data shape: {test_data.shape}")

train data shape: (891, 6)
test data shape: (418, 6)


In [ ]:
#concat_data = pd.concat([X, test_data], axis=0)

# dummy_train_data = dummy_data.iloc[:891, :]
# dummy_test_data = dummy_data.iloc[891:, :]
dummy_x = pd.get_dummies(X)
print(dummy_x.shape)
dummy_x

(891, 7)


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,3,22.000000,1,0,7.2500,0,1
1,1,38.000000,1,0,71.2833,1,0
2,3,26.000000,0,0,7.9250,1,0
3,1,35.000000,1,0,53.1000,1,0
4,3,35.000000,0,0,8.0500,0,1
...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,0,1
887,1,19.000000,0,0,30.0000,1,0
888,3,29.699118,1,2,23.4500,1,0
889,1,26.000000,0,0,30.0000,0,1


## **標準化數據**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_x)

## **搭建模型**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

## **評估模型**

In [ ]:
depth = []
for i in range(2,20,1):
  tree = DecisionTreeClassifier(criterion="entropy", max_depth = i, random_state=10)
  scores = cross_val_score(tree, sc_x, Y, cv=5)
  depth.append(scores.mean())

In [ ]:
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

7


In [ ]:
final_tree = DecisionTreeClassifier(criterion="entropy", max_depth = final_depth, random_state=10)
final_tree.fit(sc_x, Y)

DecisionTreeClassifier(criterion='entropy', max_depth=7, random_state=10)

## **測試資料產出最終數據**

In [ ]:
dummy_test_data = pd.get_dummies(test_data)
sc_test_data_x = scaler.transform(dummy_test_data)
submission = final_tree.predict(sc_test_data_x)

In [ ]:
print(submission)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 1 0 1 1 0 1 0 0 0]


In [ ]:
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission })
submission_data

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submission_data.to_csv("submission.csv", index=False)